In [9]:
import os
import csv
import json
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [10]:
class smart_dict(dict):
    @staticmethod
    def __missing__(key):
        return key

def create_save_csv(metrics=['free-response_metrics_string-match','free-response_metrics_proxy-model'], models=['gpt-4-0613','gpt-3.5-turbo-0613','text-davinci-003','text-curie-001','llama-2-70b-chat-tg','llama-2-70b-tg','flan-ul2'],pr='with'):
    model_dict = {'gpt-4-0613':'GPT4','gpt-3.5-turbo-0613':'ChatGPT','text-davinci-003':'Davinci','text-curie-001':'Curie','llama-2-70b-chat-tg':'Llama2 Chat','llama-2-70b-tg':'Llama2','flan-ul2':'Flan UL2' }
    topic_dict = smart_dict({'work/academic cheating':'Cheating','belief/ideology':'religion'})
    incentive_dict = smart_dict({'to gain financial advantage/money through bonus':'bonus', 'to gain financial advantage/money through wager':'wager'})
    metric_dict = smart_dict({'free-response_metrics_string-match':'Leakage thru. String Match','free-response_metrics_proxy-model':'Leakage thru. Proxy Agent', 'binary_metrics_error':'Binary Control Question', 'info-accessibility_metrics_error':'Information Access. Err.' , 'privacy-sharing_metrics_error': 'Private Information Access. Err.' })
    #csv_out = open(,'w')
    # writer = csv.writer(csv_out)
    # writer.writerow(['scenario_idx']+[model_dict[model] for model in models])
    df_all = pd.DataFrame(columns = ['Metric']+['Case']+[model_dict[model] for model in models])
    # Using DataFrame.insert() to add a column
    for j,metric in enumerate(metrics):
        list_avg = [f'{metric_dict[metric]}','avg']
        list_worst = [f'{metric_dict[metric]}','worst']

        for model in models:
            json_file_name = f'../../eval_results/{pr}_privacy_prompts/eval_{model}_data_tier_3_nsamples_10_q_{metric}.json'
            with open(json_file_name,'r') as jfile:
                data = json.load(jfile)
            
            
            #df_temp.loc[i] = []
            worst_case = 0
            worst_case_cnt = 0
            idx = 0
            avg_case = 0
            
            for i , s_d in enumerate(data):
                if s_d['eval_result'] :
                        worst_case = 1
                        avg_case +=1
           
                if i%10 == 9:
                    #write down:
                    worst_case_cnt += worst_case
                    idx+=1
                    worst_case = 0
                
            ## push the metrics
            list_avg.append(avg_case/len(data))
            list_worst.append(worst_case_cnt/(len(data)//10))

                    
        
        
        df_all.loc[2*j] = list_avg
        df_all.loc[2*j+1] = list_worst

    df_all.to_csv(f'../csv/tier3_table_avg_{pr}.csv')
    return df_all
            




In [11]:
def tab_to_latext(df):
    for i in range(len(df)):
        list_r = df.loc[i].to_list()[1:]
        list_r = [list_r[0].replace('_','-')] + [ '%.2f' % elem for elem in list_r[1:]]
        min_item = min(list_r[1:])
        list_row = [str(item).replace('_','-')  if item != min_item  else f"\\textbf{{{item}}}"  for item in list_r  ]
        print('& ',' & '.join(list_row),' \\\\')

# with privacy prompt

In [12]:
df_all = create_save_csv(['free-response_metrics_string-match','free-response_metrics_proxy-model','info-accessibility_metrics_error','privacy-sharing_metrics_error','binary_metrics_error'],pr='with')
df_all_sorted = df_all.sort_values(by='Case').reset_index().drop(columns=['Case','Curie']).round(2)

In [13]:
tab_to_latext(df_all_sorted)

&  Leakage thru. String Match & \textbf{0.33} & 0.71 & 0.55 & 0.75 & 0.61 & 0.62  \\
&  Leakage thru. Proxy Agent & \textbf{0.26} & 0.56 & 0.44 & 0.51 & 0.38 & 0.42  \\
&  Leakage thru. String Match & \textbf{0.54} & 0.95 & 0.88 & 1.00 & 0.99 & 0.98  \\
&  Leakage thru. Proxy Agent & \textbf{0.48} & 0.91 & 0.84 & 0.99 & 0.97 & 0.95  \\


In [15]:
df_all

,Metric,Case,GPT4,ChatGPT,Davinci,Curie,Llama2 Chat,Llama2,Flan UL2
0,Leakage thru. String Match,avg,0.331852,0.705556,0.546296,0.480000,0.747407,0.608519,0.617407
1,Leakage thru. String Match,worst,0.540741,0.951852,0.877778,0.907407,0.996296,0.992593,0.981481
2,Leakage thru. Proxy Agent,avg,0.260370,0.555185,0.441852,0.329630,0.508148,0.381852,0.417407
3,Leakage thru. Proxy Agent,worst,0.477778,0.911111,0.844444,0.837037,0.985185,0.974074,0.948148


# WO privacy prompt

In [36]:
df_all = create_save_csv(['free-response_metrics_string-match','free-response_metrics_proxy-model','info-accessibility_metrics_error','privacy-sharing_metrics_error','binary_metrics_error'],pr='without')
df_all_sorted = df_all.sort_values(by='Case').reset_index().drop(columns=['Case','Curie']).round(2)

FileNotFoundError: [Errno 2] No such file or directory: '../../eval_results/without_privacy_prompts/eval_gpt-4-0613_data_tier_3_nsamples_10_q_info-accessibility_metrics_error.json'

In [ ]:
tab_to_latext(df_all_sorted)

&  Leakage thru. String Match & \textbf{0.09} & 0.52 & 0.34 & 0.82 & 0.52 & 0.65  \\
&  Leakage thru. Proxy Agent & \textbf{0.07} & 0.40 & 0.26 & 0.53 & 0.30 & 0.46  \\
&  Information Access. Err. & \textbf{0.02} & 0.12 & 0.40 & 0.86 & 0.79 & 0.16  \\
&  Private Information Access. Err. & \textbf{0.02} & 0.09 & 0.31 & 0.83 & 0.76 & 0.12  \\
&  Binary Control Question & 0.04 & 0.01 & \textbf{0.00} & 0.39 & 0.78 & 0.36  \\
&  Leakage thru. String Match & \textbf{0.22} & 0.93 & 0.79 & 1.00 & 0.99 & 0.99  \\
&  Leakage thru. Proxy Agent & \textbf{0.20} & 0.89 & 0.74 & 0.99 & 0.96 & 0.97  \\
&  Information Access. Err. & \textbf{0.04} & 0.40 & 0.76 & 1.00 & 1.00 & 0.60  \\
&  Private Information Access. Err. & \textbf{0.03} & 0.32 & 0.70 & 1.00 & 1.00 & 0.56  \\
&  Binary Control Question & 0.06 & 0.04 & \textbf{0.00} & 0.99 & 1.00 & 0.91  \\
